In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   0.3.4
✔ tibble  3.2.1     ✔ dplyr   1.1.2
✔ tidyr   1.2.1     ✔ stringr 1.4.1
✔ readr   2.1.2     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
# grab the matching Seurat object
obj = qs::qread('_targets/objects/obj_Day5.Agrp')
obj

Loading required package: SeuratObject

Loading required package: sp

Loading required package: Seurat



An object of class Seurat 
52415 features across 2456 samples within 3 assays 
Active assay: SCT (20094 features, 3000 variable features)
 2 other assays present: RNA, integrated
 2 dimensional reductions calculated: pca, umap

In [3]:
# (n)eighbor(h)ood (ma)atrix is an object which tells you which cells belong to which neighborhoods
nhm = qs::qread('_targets/objects/nhm_Day5.Agrp___obob5v5')
nhm %>%  `[`(1:10, 1:20) %>% print

                     1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20
CTGGTCTCACCTATCC_1_1 0 0 0 0 0 0 0 0 0  0  0  0  0  0  0  0  0  0  0  0
TCATGGATCATACGAC_1_1 0 0 1 0 0 0 0 0 0  0  0  0  0  0  0  0  0  0  0  0
CTAACCCAGTCACAGG_1_1 0 0 0 0 0 0 0 0 0  0  0  0  0  0  0  1  1  0  0  0
TTGATGGCAGGCAATG_1_1 0 0 0 0 0 0 0 0 0  0  0  0  0  0  0  0  0  0  0  0
TAAGTCGTCAGACTGT_1_1 0 0 0 0 0 0 0 0 0  0  0  0  0  0  0  0  0  0  0  0
GCCGATGCATCCTGTC_1_1 0 0 0 0 0 0 0 1 0  0  0  0  0  0  0  0  0  0  0  0
CGTAAGTAGTAATACG_1_1 0 0 0 0 0 0 0 0 0  0  0  0  0  0  1  0  0  0  0  0
TTGGTTTCACTCCGGA_1_1 0 1 0 0 0 1 0 0 0  0  0  0  0  0  0  0  0  0  0  0
TCGGGACGTAATGTGA_1_1 0 0 0 0 0 0 0 0 0  0  0  0  0  0  0  0  0  0  0  0
CCCTGATTCCATCGTC_1_1 0 0 0 0 0 0 1 0 0  0  0  0  0  0  0  0  0  0  0  0


In [4]:
library(Seurat)
library(Matrix)
library(tidyverse)

# Extract Counts Matrix (ensure it's a sparse matrix)
counts_matrix <- GetAssayData(obj, slot = "counts")
# Ensure the matrix is a dgCMatrix for efficient operations
counts_matrix <- as(counts_matrix, "dgCMatrix")

# convert nhm to a dgc matrix so we can multiply it with the seurat counts dgc matrix
nhm_matrix <- as(as.matrix(nhm), "dgCMatrix")

# Matrix Multiplication
# This effectively sums the counts for each nh
summed_counts <- counts_matrix %*% nhm_matrix

# Calculate the Number of Cells per Neighborhood
cells_per_neighborhood <- colSums(nhm_matrix)

# Average Calculation
average_expression <- summed_counts / cells_per_neighborhood

# Convert to Data Frame for easier handling
average_expression_df <- as.data.frame(as.matrix(average_expression))
rownames(average_expression_df) <- rownames(counts_matrix)

# Viewing the result
head(average_expression_df)



Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack




,1,2,3,4,5,6,7,8,9,10,⋯,156,157,158,159,160,161,162,163,164,165
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Lypla1,0.44444444,0.8873239,0.6451613,0.6493506,0.7525773,0.82524272,0.5357143,0.71830986,0.921875,0.37398374,⋯,0.26865672,0.6093750,0.4862385,0.5000000,0.38461538,0.41242938,0.38888889,0.42105263,0.3773585,0.820895522
Tcea1,1.65263158,1.8913043,1.2417582,2.6326531,4.8939394,4.03846154,3.2463768,1.07453416,1.105634,1.75257732,⋯,1.57352941,1.6444444,1.2698413,0.9400000,1.01851852,1.61157025,1.40404040,1.92105263,0.4652778,2.000000000
Atp6v1h,6.63636364,11.2549020,3.8611111,14.7254902,7.7433628,8.42500000,8.5882353,4.62244898,5.893617,5.53191489,⋯,7.16326531,5.3333333,4.8888889,1.9333333,5.07812500,4.54385965,5.67857143,4.00000000,4.5468750,2.585034014
Oprk1,0.06153846,0.0125000,0.0326087,0.1212121,0.1318681,0.04761905,0.5522388,0.03658537,0.016000,0.03157895,⋯,0.03468208,0.1882353,0.5178571,0.4067797,0.02941176,0.05084746,0.05555556,0.05882353,0.1600000,0.005405405
Rb1cc1,2.08496732,2.9304348,2.1681416,7.4285714,7.8554217,4.92045455,4.7567568,3.82352941,3.132353,4.76811594,⋯,2.05405405,6.5263158,3.8513514,5.3428571,2.11650485,3.04545455,4.14893617,3.51456311,2.3333333,3.809523810
Alkal1,0.00000000,0.0000000,0.0000000,0.0000000,0.0000000,0.00000000,0.0000000,0.00000000,0.000000,0.00000000,⋯,0.00000000,0.0000000,0.0000000,0.0000000,0.00000000,0.00000000,0.00000000,0.00000000,0.0000000,0.000000000


In [5]:
degs = qs::qread('_targets/objects/combined_deg_seurat_formatted')

In [6]:
degs %>%
filter(cluster == 'Agrp') %>%
filter(fgf1_comparison == 'obob14v14') %>%
filter(bl6_comparison == 'obobBL6d14') %>%
filter(grouping == 'restored_grouping') %>%
filter(cells_a == 'neg_restored') %>%
filter(cells_b == 'none') %>%
filter(fgf1_day == 'Day14') %>%
filter(bl6_day == 'Day14') %>%
filter(avg_log2FC > 0) %>% # I only counted genes with avg_log2FC > 0 in the other charts %>%
head(10)

tag,data_day,cluster,fgf1_day,fgf1_comparison,bl6_day,bl6_comparison,grouping,cells_a,cells_b,GeneID,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,ensmusg,gsea_sort_score
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
Agrp___Day14.obob14v14__v__Day14.obobBL6d14___restored_grouping.neg_restored_vs_none,all,Agrp,Day14,obob14v14,Day14,obobBL6d14,restored_grouping,neg_restored,none,Pde10a,1.257125e-58,0.9274343,1.000,0.976,2.473393e-54,ENSMUSG00000023868,49.71670
Agrp___Day14.obob14v14__v__Day14.obobBL6d14___restored_grouping.neg_restored_vs_none,all,Agrp,Day14,obob14v14,Day14,obobBL6d14,restored_grouping,neg_restored,none,Sorcs1,1.435887e-43,1.2561620,0.922,0.674,2.825109e-39,ENSMUSG00000043531,48.42375
Agrp___Day14.obob14v14__v__Day14.obobBL6d14___restored_grouping.neg_restored_vs_none,all,Agrp,Day14,obob14v14,Day14,obobBL6d14,restored_grouping,neg_restored,none,Ptchd4,6.572679e-40,1.2066947,0.511,0.188,1.293175e-35,ENSMUSG00000042256,42.09958
Agrp___Day14.obob14v14__v__Day14.obobBL6d14___restored_grouping.neg_restored_vs_none,all,Agrp,Day14,obob14v14,Day14,obobBL6d14,restored_grouping,neg_restored,none,ENSMUSG00000118257,5.545884e-51,0.8409582,0.997,0.852,1.091153e-46,ENSMUSG00000118257,38.65222
Agrp___Day14.obob14v14__v__Day14.obobBL6d14___restored_grouping.neg_restored_vs_none,all,Agrp,Day14,obob14v14,Day14,obobBL6d14,restored_grouping,neg_restored,none,Rgs20,4.330114e-40,1.0847010,0.761,0.441,8.519499e-36,ENSMUSG00000002459,38.04002
Agrp___Day14.obob14v14__v__Day14.obobBL6d14___restored_grouping.neg_restored_vs_none,all,Agrp,Day14,obob14v14,Day14,obobBL6d14,restored_grouping,neg_restored,none,Cdh13,6.519334e-38,1.0644803,0.806,0.498,1.282679e-33,ENSMUSG00000031841,35.01276
Agrp___Day14.obob14v14__v__Day14.obobBL6d14___restored_grouping.neg_restored_vs_none,all,Agrp,Day14,obob14v14,Day14,obobBL6d14,restored_grouping,neg_restored,none,Asap1,1.971465e-50,0.6750550,1.000,0.995,3.878857e-46,ENSMUSG00000022377,30.65512
Agrp___Day14.obob14v14__v__Day14.obobBL6d14___restored_grouping.neg_restored_vs_none,all,Agrp,Day14,obob14v14,Day14,obobBL6d14,restored_grouping,neg_restored,none,Marchf11,9.320740e-38,0.9281514,0.841,0.506,1.833856e-33,ENSMUSG00000022269,30.38455
Agrp___Day14.obob14v14__v__Day14.obobBL6d14___restored_grouping.neg_restored_vs_none,all,Agrp,Day14,obob14v14,Day14,obobBL6d14,restored_grouping,neg_restored,none,Eda,1.752577e-36,0.9501833,0.935,0.674,3.448196e-32,ENSMUSG00000059327,29.89505


In [7]:
# filter to the NMGs for Agrp, Day14, restored_grouping, neg_restored, avg_log2FC >0

these_degs = degs %>%
filter(cluster == 'Agrp') %>%
filter(fgf1_comparison == 'obob14v14') %>%
filter(bl6_comparison == 'obobBL6d14') %>%
filter(grouping == 'restored_grouping') %>%
filter(cells_a == 'neg_restored') %>%
filter(cells_b == 'none') %>%
filter(fgf1_day == 'Day14') %>%
filter(bl6_day == 'Day14') %>%
filter(avg_log2FC > 0) %>% # I only counted genes with avg_log2FC > 0 in the other charts
pull(GeneID)

these_degs %>% head

[1] "Pde10a"             "Sorcs1"             "Ptchd4"            
[4] "ENSMUSG00000118257" "Rgs20"              "Cdh13"

In [8]:
# da_results_nhg - da results with neighborhood grouping 
# this object gives you what "grouping" neighborhoods fall under depending on the logic conditions met
# there is a restored_grouping, fgf1_grouping, and bl6 grouping column one can select from
da_results_nhg = qs::qread('_targets/objects/da_results_nhg_Agrp___Day5.obob5v5__v__Day5.obobBL6d5')
da_results_nhg %>% head

,logFC,logCPM,F,PValue,FDR,Nhood,SpatialFDR,polarity,labels,labels_fraction,comparison,restored,exact_grouping,restored_grouping,fgf1_grouping,bl6_grouping
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>
1,4.255692,12.48025,23.85359,1.267251e-06,0.0002090965,31,0.0002053307,pos,Agrp,1,Day5.obob5v5__v__Day5.obobBL6d5,FALSE,pos_FGF1,pos_FGF1,pos,none
2,-3.626020,12.84004,21.90276,4.341997e-06,0.0003582147,56,0.0003438041,neg,Agrp,1,Day5.obob5v5__v__Day5.obobBL6d5,TRUE,neg_restored,neg_restored,neg,pos
3,3.074480,12.93484,18.90275,1.562452e-05,0.0003682922,10,0.0003808797,pos,Agrp,1,Day5.obob5v5__v__Day5.obobBL6d5,TRUE,pos_restored,pos_restored,pos,neg
4,-3.128057,12.90771,19.22492,1.325897e-05,0.0003682922,20,0.0003808797,neg,Agrp,1,Day5.obob5v5__v__Day5.obobBL6d5,TRUE,neg_restored,neg_restored,neg,pos
5,-3.101828,12.77347,19.06553,1.438396e-05,0.0003682922,55,0.0003808797,neg,Agrp,1,Day5.obob5v5__v__Day5.obobBL6d5,TRUE,neg_restored,neg_restored,neg,pos
6,3.785717,12.82842,19.03564,1.459809e-05,0.0003682922,68,0.0003808797,pos,Agrp,1,Day5.obob5v5__v__Day5.obobBL6d5,TRUE,pos_restored,pos_restored,pos,neg


In [9]:
# I pull all the neg_restored nh from restored_grouping
restored_nhoods = da_results_nhg %>%
filter(restored_grouping == 'neg_restored') %>%
mutate(Nhood = as.character(as.integer(Nhood))) %>% 
pull(Nhood)
restored_nhoods

[1] "56"  "20"  "55"  "112" "69"  "116" "3"   "67"  "133" "76"  "93"  "111"
[13] "27"  "138" "44"  "102" "22"  "118" "137"

In [10]:
# here I also pull some restored nh to compare against
none_nhoods = da_results_nhg %>%
filter(restored_grouping == 'none') %>%
mutate(Nhood = as.character(as.integer(Nhood))) %>% 
# sample_n(length(restored_nhoods)) %>% # optionally you can randomly sample an equal amount of nones, or pull all of them by commenting this line out
pull(Nhood) 

none_nhoods

[1] "17"  "124" "125" "155" "9"   "129" "2"   "89"  "150" "97"  "40"  "87" 
 [13] "120" "122" "1"   "7"   "117" "83"  "49"  "85"  "94"  "100" "114" "134"
 [25] "140" "161" "165" "119" "58"  "59"  "132" "36"  "99"  "143" "110" "18" 
 [37] "26"  "28"  "147" "72"  "88"  "77"  "153" "13"  "8"   "70"  "90"  "98" 
 [49] "106" "139" "145" "152" "154" "163" "92"  "82"  "65"  "12"  "96"  "57" 
 [61] "14"  "105" "15"  "113" "151" "74"  "103" "130" "142" "23"  "48"  "115"
 [73] "159" "46"  "141" "4"   "32"  "61"  "63"  "80"  "95"  "127" "64"  "81" 
 [85] "109" "157" "38"  "126" "6"   "121" "146" "53"  "101" "5"   "11"  "51" 
 [97] "16"  "75"  "135" "78"  "164" "156" "39"  "21"  "24"  "29"  "30"  "33" 
[109] "34"  "35"  "37"  "42"  "43"  "47"  "54"  "60"  "62"  "71"  "73"  "86" 
[121] "107" "108" "136" "144" "148" "149" "158" "162"

In [11]:
# I select restored nhoods and none_nhoods to become columns in a dataframe later
selected_nhoods = c(restored_nhoods, none_nhoods)

In [12]:
# this is the NMGs x selected_nhoods dataframe 
# on visual inspection, the columns to the left (neg_restored nh) have higher expression of these genes than the columns to the right (none nh)
heatmap_df = average_expression_df[these_degs, selected_nhoods]
heatmap_df %>% dim
heatmap_df %>% head

[1] 436 147

,56,20,55,112,69,116,3,67,133,76,⋯,73,86,107,108,136,144,148,149,158,162
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Pde10a,69.961538,83.680556,57.659091,15.1016949,44.535211,46.298507,43.211111,41.718447,58.483871,55.066667,⋯,3.55000000,112.642857,5.68656716,5.23437500,18.238806,12.5084746,4.1891892,2.81538462,29.7368421,6.9104478
Sorcs1,21.388060,13.437500,57.018868,6.4444444,19.094828,17.185567,25.070423,20.241379,37.062500,29.626866,⋯,0.93548387,48.073529,1.24858757,1.14444444,9.232143,2.1076923,3.5054945,2.21428571,9.4375000,0.7514124
Ptchd4,1.121739,1.232143,4.265823,0.2815534,5.469388,4.435484,1.567901,1.024096,2.156463,7.290698,⋯,0.06250000,1.551020,0.02083333,0.01941748,0.712963,0.2258065,0.2333333,0.02898551,0.4603175,0.0625000
ENSMUSG00000118257,18.407407,12.991304,22.313953,10.6666667,16.072727,12.214876,9.375839,18.685714,19.493976,19.513274,⋯,0.88095238,40.932203,2.40625000,1.15217391,6.519608,3.8095238,6.1717172,1.55263158,12.2898551,2.3125000
Rgs20,2.209302,1.758333,4.642857,1.3125000,3.435484,1.703704,1.360544,1.882979,5.301887,5.954545,⋯,0.16666667,3.487500,0.12500000,0.32812500,1.155963,1.3239437,1.7283951,0.30841121,2.1341463,0.1041667
Cdh13,3.560976,1.063830,7.132530,2.0204082,3.453704,8.656250,4.632184,1.803030,8.272727,15.776596,⋯,0.08235294,4.075472,0.22950820,0.14285714,4.588889,0.7522936,0.6687898,0.31250000,2.6465517,0.3770492


In [13]:
# we can also average these out to compare them at a glance

In [14]:
neg_restored_exp = average_expression_df[these_degs,restored_nhoods] %>%
rowMeans

In [15]:
none_exp = average_expression_df[these_degs,none_nhoods] %>%
rowMeans

In [16]:
# here is the comparison. It mostly holds - we expect all NMGs to have a positive diff
data.frame(neg_restored_exp, none_exp) %>%
mutate(diff = neg_restored_exp - none_exp) %>%
head(20)

,neg_restored_exp,none_exp,diff
,<dbl>,<dbl>,<dbl>
Pde10a,48.203859,40.461010,7.7428498
Sorcs1,24.909742,16.505933,8.4038082
Ptchd4,2.834914,1.299600,1.5353143
ENSMUSG00000118257,16.076380,13.984671,2.0917091
Rgs20,2.720682,2.159467,0.5612149
Cdh13,5.890497,3.125148,2.7653487
Asap1,24.160912,17.060894,7.1000184
Marchf11,6.178188,4.147166,2.0310221
Eda,12.896916,11.091946,1.8049707


In [17]:
# not all NMGs have a positive difference because this was computed on cells, not neighborhoods
# at one point, I use a voting strategy described in the manuscript to assign cells to neighborhoods exclusively
# that's kinda necessary to make seurat's FindMarkers work
# but we can also examine this on the level of cells that got the neg_restored and pos_restored labels

In [18]:
# nhgc - (n)eighbor(h)ood (g)rouping by (c)ell
# gives the vote outcome for each grouping col, now applied to cells
nhgc = qs::qread('_targets/objects/nhgc_Agrp___Day5.obob5v5__v__Day5.obobBL6d5')
nhgc %>% head

rowname,restored_grouping,bl6_grouping,fgf1_grouping
<chr>,<fct>,<chr>,<chr>
ATGGATCAGCATCAAA_5_2,pos_FGF1,none,pos
TTCCTTCGTTGTCAGT_2_3,pos_FGF1,none,pos
TACCGAAAGTGAGGCT_3_3,pos_FGF1,none,pos
ACTCCCAGTCTCTCCA_5_3,pos_FGF1,none,pos
TCGTAGAAGCACTCTA_2_1,pos_restored,neg,pos
ACCAAACGTCGATTAC_3_1,pos_restored,neg,pos


In [19]:
# select cells_a (neg_restored in this case) and cells_b (none)
cells_a = nhgc %>%
filter(restored_grouping == 'neg_restored') %>%
pull(rowname)

cells_b = nhgc %>% 
filter(restored_grouping == 'none') %>%
pull(rowname)



In [20]:
# select cells_a and cells_b
selected_cells = c(cells_a, cells_b)
# make cell_heatmap_df
cell_heatmap_df = counts_matrix[these_degs, selected_cells] %>%
as.data.frame

In [21]:
# let's look at means
exp_a = counts_matrix[these_degs,cells_a] %>% rowMeans
exp_b = counts_matrix[these_degs,cells_b] %>% rowMeans

data.frame(exp_a, exp_b) %>%
mutate(diff = exp_a - exp_b) %>% 
head(20)

,exp_a,exp_b,diff
,<dbl>,<dbl>,<dbl>
Pde10a,51.642857,31.3478261,20.29503106
Sorcs1,22.355159,13.6548090,8.70034977
Ptchd4,2.490079,0.9822134,1.50786593
ENSMUSG00000118257,17.509921,11.5704875,5.93943315
Rgs20,2.805556,1.7786561,1.02689943
Cdh13,6.398810,2.5955204,3.80328910
Asap1,23.545635,13.7509881,9.79464678
Marchf11,6.849206,2.9341238,3.91508250
Eda,12.384921,8.7167325,3.66818809
